<img src="photo.jpg"/>

In [5]:
// Create the space for the full gene:
float [] gene = new float[200];

// Set each location to a number between 0 and 1:
for (int i = 0; i < 200; i++) {
    gene[i] = random(1);
}

// Create a place to draw the creature (the Phenotype):
PGraphics pg = createGraphics(100, 100);

// Now we draw it:
for (int i = 0; i < 200; i += 2) {
    float r = 0;
    float g = 0;
    float b = 0;
    float a = 255;
    float radius = 5;
    float x = gene[i] * 100;
    float y = gene[i + 1] * 100;
    pg.fill(r, g, b, a);
    pg.noStroke();
    pg.ellipse(x, y, radius, radius);
}
// Finally, show the creature:
image(pg, 0, 0); 

<IPython.core.display.Javascript object>

In [3]:
%%processing
int POPSIZE = 25;
int GENESIZE = 100 * (2); // 100 individuals * (x+y) 

PImage target = loadImage("photo.jpg");
target.loadPixels();

float [][] pop = new float[POPSIZE][GENESIZE];
float [] fitness = new float[POPSIZE];
PGraphics [] pgraphics = new PGraphics[POPSIZE];

// Gene creation (random)
float[] makeGene() {
    float[] gene = new float[GENESIZE];
    for (int i = 0; i < GENESIZE; i++) {
        gene[i] = random(.999999);
    }
    return gene;
}

float[] copyGene(int pos) {
    float[] gene = new float[GENESIZE];
    for (int i = 0; i < GENESIZE; i++) {
        gene[i] = pop[pos][i];
    }
    return gene;
}

//Init population
void setup() {
    // Create the population:
    size(500, 500);
    for (int i = 0; i < POPSIZE; i++) {
        pop[i] = makeGene();
        pgraphics[i] = createGraphics(100, 100);
    }
}

// Compute genome of an individal 
PGraphics createPhenotype(int gene_pos, int w, int h) {
    pg = pgraphics[gene_pos];
    pg.background(255);
    for (int i = 0; i < GENESIZE; i += 2) {
        r = 0;
        g = 0;
        b = 0;
        a = 255;
        radius = 5;        
        x = pop[gene_pos][i] * w;
        y = pop[gene_pos][i + 1] * h;
        pg.fill(r, g, b, a);
        pg.noStroke();
        pg.ellipse(x, y, radius, radius);
    }
    return pg;
}

// Draw individuals
void drawPopulation() {
    gene = 0;
    for (j=0; j < 5; j++) {
        for (i=0; i < 5; i++) {
            pg = createPhenotype(gene, 100, 100);
            image(pg, i * 100, j * 100); 
            fill(0);
            text(fitness[gene], i * 100 + 1, j * 100 + 100 + 1); 
            fill(255);
            text(fitness[gene], i * 100, j * 100 + 100); 
            gene++;
        }
    }
}

// Population sorting for display
void sortPopulation() {
    for (int i = 0; i < POPSIZE - 1; i++) {
        for (int j = i + 1; j < POPSIZE; j++) {
            if (fitness[i] < fitness[j]) { // swap
                temp_fitness = fitness[i];
                fitness[i] = fitness[j];
                fitness[j] = temp_fitness;
                // swap genes:
                for (int c = 0; c < GENESIZE; c++) {
                    temp = pop[i][c];
                    pop[i][c] = pop[j][c];
                    pop[j][c] = temp;
                }
            }
        }
    }
}

// Fitness = score calculation
int computeFitness(int gene) {
    pg = createPhenotype(gene, 100, 100);
    gdata = pg.toImageData().data; // height, width, data
    tdata = target.imageData.data;
    fit = 0;
    for (int i = 0; i < (100 * 100 * 4); i++) {
        fit += abs(gdata[i] - tdata[i]);
    }
    return 100 * 100 * 4 * 255 - fit;
}

// Mutation 
void mutatePopulation(percent) {
    for (int p = 1; p < POPSIZE; p++) {
        for (int i = 0; i < GENESIZE; i++) {
            if (random(1) < percent) 
                pop[p][i] = random(0, .99999);
        }
    }
}

int fitnessSum() {
    sum = 0;
    for (int i = 0; i < POPSIZE; i++) {
        sum += fitness[0];
    }
    return sum;
}

int select() {
    index = 0;
    partsum = 0.0;
    sumFitness = fitnessSum();
    spin = random() * sumFitness;
    while (index < POPSIZE - 1) {
        partsum += fitness[index];
        if (partsum >= spin) {
            break;
        }
        index += 1;
    }
    return index;
}

// Crossover
void crossoverPopulation(percent) {
    for (int p = POPSIZE - 1; p > POPSIZE - (POPSIZE * percent); p--) {
        c1 = select();
        c2 = select();
        crossover_point = random(GENESIZE);
        for (int i = 0; i < GENESIZE; i++) {
            if (i < crossover_point) 
                pop[p][i] = pop[c1][i];
            else
                pop[p][i] = pop[c2][i];
        }
    }
}

void draw() {
    for (int p = 0; p < POPSIZE; p++) {
        fitness[p] = computeFitness(p);
    }
    sortPopulation();
    drawPopulation();
    mutatePopulation(.01);
    crossoverPopulation(.80);
}
